<a href="https://colab.research.google.com/github/real-rookie/novelty-detection-algorithms-evaluation/blob/main/generic_one_to_many.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# env
!pip install lightning
!pip install anomalib
!pip install OpenVINO
!pip install wandb

In [ ]:
# unzip code and datasets
!unzip -o /content/drive/MyDrive/novelty-detection-algorithms-evaluation.zip -d /home/
%cd /home

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.utils import save_image
import os
import random
import numpy as np

In [ ]:
# set parameters
DATASET_INFO = {
    # idx 0: paths, idx 1: number of classes
    "MNIST": ["datasets/MNIST/images", 10],
    "FashionMNIST": ["datasets/FashionMNIST/images", 10],
    "CIFAR10": ["datasets/CIFAR10/images", 10],
}
dataset = "MNIST"
dataset_path = DATASET_INFO[dataset][0]
num_total_classes = DATASET_INFO[dataset][1]
normal_weight = 0.5 # proportion of normal samples in the test sets

In [ ]:
# make datasets
%cd /home
os.system(f"rm -rf {dataset_path}")
for i in range(num_total_classes):
    os.system(f"mkdir -p {dataset_path}/train/{i}")
    os.system(f"mkdir -p {dataset_path}/categorized_test_cases/{i}")
    os.system(f"mkdir -p {dataset_path}/test/{i}/normal")
    os.system(f"mkdir -p {dataset_path}/test/{i}/novel")

/home


In [ ]:
train_data = None
test_data = None
if dataset == "MNIST":
    train_data = datasets.MNIST(root="datasets", train=True, download=True, transform=ToTensor())
    test_data = datasets.MNIST(root="datasets", train=False, download=True, transform=ToTensor())
elif dataset == "FashionMNIST":
    train_data = datasets.FashionMNIST(root="datasets", train=True, download=True, transform=ToTensor())
    test_data = datasets.FashionMNIST(root="datasets", train=False, download=True, transform=ToTensor())
elif dataset == "CIFAR10":
    train_data = datasets.CIFAR10(root="datasets/CIFAR10", train=True, download=True, transform=ToTensor())
    test_data = datasets.CIFAR10(root="datasets/CIFAR10", train=False, download=True, transform=ToTensor())
else:
    print("Wrong dataset specified")
    os.abort()

In [ ]:
train_counter = np.zeros(num_total_classes, dtype=int)
test_counter = np.zeros(num_total_classes, dtype=int)
for img, label in train_data:
    save_image(img, f"{dataset_path}/train/{label}/{label}_{train_counter[label]}.png")
    train_counter[label] += 1
for img, label in test_data:
    save_image(img, f"{dataset_path}/categorized_test_cases/{label}/{label}_{test_counter[label]}.png")
    test_counter[label] += 1
print(f"train: {train_counter}")
print(f"test: {test_counter}")

train: [5923 6742 5958 6131 5842 5421 5918 6265 5851 5949]
test: [ 980 1135 1032 1010  982  892  958 1028  974 1009]


In [ ]:
def fill_test_sets(population, num_samples, src_cls, dest_cls):
    sample_idx = random.sample(range(population), num_samples)
    folder_type = "normal" if src_cls == dest_cls else "novel"
    src_set = None
    dest_set = None
    for index in sample_idx:
            os.system(f"cp {dataset_path}/categorized_test_cases/{src_cls}/{src_cls}_{index}.png {dataset_path}/test/{dest_cls}/{folder_type}")

for normal in range(num_total_classes):
    random.seed(normal)

    # test normal
    num_normal_test_samples = np.floor(test_counter[normal] * normal_weight).astype(int)
    fill_test_sets(test_counter[normal], num_normal_test_samples, normal, normal)

    num_novel_test_from_each_class = np.floor((test_counter[normal] - num_normal_test_samples) / (num_total_classes - 1)).astype(int)
    for novel in range(num_total_classes):
        if(novel == normal):
            continue
        # test novel
        fill_test_sets(test_counter[novel], num_novel_test_from_each_class, novel, normal)


In [ ]:
# training and testing
%cd /home/novelty-detection-algorithms-evaluation
!python generic_one_to_many.py --mode train --data MNIST --model RD4AD

/home/novelty-detection-algorithms-evaluation
/usr/local/lib/python3.10/dist-packages/anomalib/config/config.py:238: UserWarning: The seed value is now fixed to 0. Up to v0.3.7, the seed was not fixed when the seed value was set to 0. If you want to use the random seed, please select `None` for the seed value (`null` in the YAML file) or remove the `seed` key from the YAML file.
  warn(
/usr/local/lib/python3.10/dist-packages/anomalib/config/config.py:275: UserWarning: config.project.unique_dir is set to False. This does not ensure that your results will be written in an empty directory and you may overwrite files.
  warn(
Global seed set to 0
2023-07-26 23:19:45,920 - anomalib.data - INFO - Loading the datamodule
2023-07-26 23:19:45,921 - anomalib.data.utils.transform - INFO - No config file has been provided. Using default transforms.
2023-07-26 23:19:45,921 - anomalib.data.utils.transform - INFO - No config file has been provided. Using default transforms.
2023-07-26 23:19:45,921 - 

In [ ]:
!python generic_one_to_many.py --mode test --data MNIST --model RD4AD

/usr/local/lib/python3.10/dist-packages/anomalib/config/config.py:238: UserWarning: The seed value is now fixed to 0. Up to v0.3.7, the seed was not fixed when the seed value was set to 0. If you want to use the random seed, please select `None` for the seed value (`null` in the YAML file) or remove the `seed` key from the YAML file.
  warn(
/usr/local/lib/python3.10/dist-packages/anomalib/config/config.py:275: UserWarning: config.project.unique_dir is set to False. This does not ensure that your results will be written in an empty directory and you may overwrite files.
  warn(
/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
FeatureExtractor is deprecated. Use TimmFeatureExtractor instead. Both FeatureExtractor and TimmFeatureExtractor will be removed in a future release.
/us